<a href="https://colab.research.google.com/github/cesar-ramosh/Gobernanzas-de-datos/blob/main/Sector_Precios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gobernanza de datos - Sector Precios

In [1]:
# Enlazar Google Drive con Google Colaboratory - Notebook
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
%cd /content/drive/MyDrive/Sector Precios/

Mounted at /content/drive
/content/drive/MyDrive/Sector Precios


In [2]:
%cd /content/drive/MyDrive/Sector Precios/

/content/drive/MyDrive/Sector Precios


# Importación de datos

In [57]:
# Cargar la segunda hoja en un DataFrame de Pandas y utilizar la primera columna como índice de fechas
# 398 productos
IPC_Productos = pd.read_excel('01. Base Sector Precios.xlsx', sheet_name='Bolivia_Productos')
#IPC_Productos = pd.read_excel('01. Base Sector Precios.xlsx', sheet_name='Bolivia_Productos', index_col=0)
# 13 variables divisiones
IPC_Division = pd.read_excel('01. Base Sector Precios.xlsx', sheet_name='Divisiones')
#IPC_Division = pd.read_excel('01. Base Sector Precios.xlsx', sheet_name='Divisiones', index_col=0)
# 9 departamentos + Nacional
IPC_Ciudad = pd.read_excel('01. Base Sector Precios.xlsx', sheet_name='Ciudades')
# 3 variables
IPC_Alimento = pd.read_excel('01. Base Sector Precios.xlsx', sheet_name='Alimentos')
#IPC_Alimento = pd.read_excel('01. Base Sector Precios.xlsx', sheet_name='Alimentos', index_col=0)
IPC_Historical = pd.read_excel('01. Base Sector Precios.xlsx', sheet_name='ipc_nal_histo')
#IPC_Alimento = pd.read_excel('01. Base Sector Precios.xlsx', sheet_name='Alimentos', index_col=0)

,mes,IPC_2016,IPC_1991
0,1937-01-31,1.423620e-08,6.432980e-08
1,1937-02-28,1.458770e-08,6.591820e-08
2,1937-03-31,1.537860e-08,6.949210e-08
3,1937-04-30,1.432410e-08,6.472690e-08
4,1937-05-31,1.406050e-08,6.353560e-08
...,...,...,...
1036,2023-05-31,1.094398e+02,4.945503e+02
1037,2023-06-30,1.096786e+02,4.956295e+02
1038,2023-07-31,1.100817e+02,4.974511e+02
1039,2023-08-31,1.105068e+02,4.993722e+02


In [4]:
# Ponderadores de cada actividad
w0_divis = pd.read_excel('04. Ponderaciones.xlsx', sheet_name='Divisiones')
w0_depto = pd.read_excel('04. Ponderaciones.xlsx', sheet_name='Ciudad')
w0_alimen = pd.read_excel('04. Ponderaciones.xlsx', sheet_name='Alimento_No')
w0_produc = pd.read_excel('04. Ponderaciones.xlsx', sheet_name='Productos')
#suma = w0_alimen['W0'].sum()

## Función para el cálculo de la inflación mensual, a 12 meses y acumulada

In [5]:
import pandas as pd

def calcular_inflacion(df, columna_variable, columna_fecha):
    # Hacer una copia del DataFrame para evitar modificar el original
    df = df.copy()

    # Calcular la tasa de crecimiento mensual
    df[f'imes_{columna_variable}'] = df[columna_variable].pct_change() * 100

    # Calcular la tasa de crecimiento a 12 meses
    df[f'i12m_{columna_variable}'] = df[columna_variable].pct_change(periods=12) * 100

    # Convertir la columna de fecha a formato "datetime" si no está en ese formato
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])

    # Extraer el año de la columna de fecha
    df['year'] = df[columna_fecha].dt.year

    # Crear la función para el año de referencia
    min_year = df['year'].min()
    max_year = df['year'].max()

    for year in range(min_year, max_year):
        base_col = f"base_{year}"
        df[base_col] = df.loc[(df['year'] == year) & (df[columna_fecha].dt.month == 12), columna_variable].values[0]

    # Calcular la tasa de crecimiento acumulada utilizando el enfoque de referencia anual
    for year in range(min_year + 1, max_year + 1):
        prev_year = year - 1
        df.loc[df['year'] == year, f'icum_{columna_variable}'] = ((df[columna_variable] / df[f"base_{prev_year}"]) - 1) * 100

    # Eliminar las variables auxiliares
    df = df.drop(['year'] + [f"base_{year}" for year in range(min_year, max_year)], axis=1)

    return df

# Índice Precios al Consumidor por departamento

## Limpieza de datos

In [6]:
import pandas as pd
# Eliminar observaciones que contienen valores nulos how=all
#IPC_Ciudad = IPC_Ciudad.dropna(axis=0)
IPC_Ciudad = IPC_Ciudad.dropna(how='all')
IPC_Ciudad.head(2)

,mes,Bolivia,Sucre,La Paz,Cochabamba,Oruro,Potosí,Tarija,Santa Cruz,Beni,Pando
0,1993-01-01,26.163202,NaN,28.874885,21.911197,NaN,NaN,NaN,25.606815,NaN,NaN
1,1993-02-01,26.412003,NaN,29.372096,22.088338,NaN,NaN,NaN,25.646317,NaN,NaN


## Cálculo inflación IPC por ciudades

In [7]:
Infla_bol = calcular_inflacion(IPC_Ciudad, 'Bolivia', 'mes')
Infla_bol = Infla_bol.iloc[:, [0, 11, 12, 13]]
Infla_scr = calcular_inflacion(IPC_Ciudad, 'Sucre', 'mes')
Infla_scr = Infla_scr.iloc[:, [11, 12, 13]]
Infla_lpz = calcular_inflacion(IPC_Ciudad, 'La Paz', 'mes')
Infla_lpz = Infla_lpz.iloc[:, [11, 12, 13]]
Infla_cba = calcular_inflacion(IPC_Ciudad, 'Cochabamba', 'mes')
Infla_cba = Infla_cba.iloc[:, [11, 12, 13]]
Infla_oru = calcular_inflacion(IPC_Ciudad, 'Oruro', 'mes')
Infla_oru = Infla_oru.iloc[:, [11, 12, 13]]
Infla_pot = calcular_inflacion(IPC_Ciudad, 'Potosí', 'mes')
Infla_pot = Infla_pot.iloc[:, [11, 12, 13]]
Infla_tar = calcular_inflacion(IPC_Ciudad, 'Tarija', 'mes')
Infla_tar = Infla_tar.iloc[:, [11, 12, 13]]
Infla_scz = calcular_inflacion(IPC_Ciudad, 'Santa Cruz', 'mes')
Infla_scz = Infla_scz.iloc[:, [11, 12, 13]]
Infla_ben = calcular_inflacion(IPC_Ciudad, 'Beni', 'mes')
Infla_ben = Infla_ben.iloc[:, [11, 12, 13]]
Infla_pdo = calcular_inflacion(IPC_Ciudad, 'Pando', 'mes')
Infla_pdo = Infla_pdo.iloc[:, [11, 12, 13]]
# Concatenar los DataFrames en uno solo
infla_depto = pd.concat([Infla_bol, Infla_scr, Infla_lpz, Infla_cba, Infla_oru, Infla_pot, Infla_tar, Infla_scz, Infla_ben, Infla_pdo], axis=1)
import pandas as pd
# Eliminar observaciones que contienen valores nulos
#infla_depto = infla_depto.dropna(axis=0)
infla_depto = infla_depto.dropna(how='all')
# Formato de fechas
infla_depto['mes'] = infla_depto['mes'].dt.strftime('%b-%y')
# Redondear los decimales
infla_depto = infla_depto.round(10)

# Índice Precios al Consumidor por división

## Limpieza de datos

In [8]:
import pandas as pd
# Eliminar observaciones que contienen valores nulos
#IPC_Division = IPC_Division.dropna(axis=0)
IPC_Division = IPC_Division.dropna(how='all')
IPC_Division.head(2)

,mes,Bolivia,Alimentos y bebidas no alcohólicas,Bebidas alcohólicas y tabaco,Prendas de vestir y calzado,Vivienda y servicios básicos,"Muebles, bienes y servicios domésticos",Salud,Transporte,Comunicaciones,Recreación y cultura,Educación,Alimentos y bebidas consumidos fuera del hogar,Bienes y servicios diversos
0,1992-03-01,24.551016,20.787881,NaN,35.001393,26.626877,23.769239,20.629507,NaN,NaN,31.287264,25.521613,NaN,31.305815
1,1992-04-01,24.615680,20.773083,NaN,35.110960,27.021873,23.855080,20.909262,NaN,NaN,31.496949,25.783236,NaN,31.641182


## Cálculo inflación por división

In [9]:
Infla_bol = calcular_inflacion(IPC_Division, 'Bolivia', 'mes')
Infla_bol = Infla_bol.iloc[:, [0, 14, 15, 16]]
Infla_abnh = calcular_inflacion(IPC_Division, 'Alimentos y bebidas no alcohólicas', 'mes')
Infla_abnh = Infla_abnh.iloc[:, [14, 15, 16]]
Infla_bayt = calcular_inflacion(IPC_Division, 'Bebidas alcohólicas y tabaco', 'mes')
Infla_bayt = Infla_bayt.iloc[:, [14, 15, 16]]
Infla_pvyc = calcular_inflacion(IPC_Division, 'Prendas de vestir y calzado', 'mes')
Infla_pvyc = Infla_pvyc.iloc[:, [14, 15, 16]]
Infla_vysb = calcular_inflacion(IPC_Division, 'Vivienda y servicios básicos', 'mes')
Infla_vysb = Infla_vysb.iloc[:, [14, 15, 16]]
Infla_mbsd = calcular_inflacion(IPC_Division, 'Muebles, bienes y servicios domésticos', 'mes')
Infla_mbsd = Infla_mbsd.iloc[:, [14, 15, 16]]
Infla_salu = calcular_inflacion(IPC_Division, 'Salud', 'mes')
Infla_salu = Infla_salu.iloc[:, [14, 15, 16]]
Infla_tran = calcular_inflacion(IPC_Division, 'Transporte', 'mes')
Infla_tran = Infla_tran.iloc[:, [14, 15, 16]]
Infla_comu = calcular_inflacion(IPC_Division, 'Comunicaciones', 'mes')
Infla_comu = Infla_comu.iloc[:, [14, 15, 16]]
Infla_reyc = calcular_inflacion(IPC_Division, 'Recreación y cultura', 'mes')
Infla_reyc = Infla_reyc.iloc[:, [14, 15, 16]]
Infla_educ = calcular_inflacion(IPC_Division, 'Educación', 'mes')
Infla_educ = Infla_educ.iloc[:, [14, 15, 16]]
Infla_acfh = calcular_inflacion(IPC_Division, 'Alimentos y bebidas consumidos fuera del hogar', 'mes')
Infla_acfh = Infla_acfh.iloc[:, [14, 15, 16]]
Infla_bysd = calcular_inflacion(IPC_Division, 'Bienes y servicios diversos', 'mes')
Infla_bysd = Infla_bysd.iloc[:, [14, 15, 16]]
# Concatenar los DataFrames en uno solo
infla_divis = pd.concat([Infla_bol, Infla_abnh, Infla_bayt, Infla_pvyc, Infla_vysb, Infla_mbsd, Infla_salu, Infla_tran, Infla_comu, Infla_reyc, Infla_educ, Infla_acfh, Infla_bysd], axis=1)
import pandas as pd
# Eliminar observaciones que contienen valores nulos
#infla_divis = infla_divis.dropna(axis=0)
infla_divis = infla_divis.dropna(how='all')
# Formato de fechas
#infla_divis['mes'] = infla_divis['mes'].dt.strftime('%b-%y')
infla_divis.loc[:, 'mes'] = infla_divis['mes'].dt.strftime('%b-%y')
# Redondear los decimales
infla_divis = infla_divis.round(10)
#infla_divis = infla_divis.round(2)

# Índice Precios al Consumidor - Alimentos

## Limpieza de datos

In [43]:
import pandas as pd
# Eliminar observaciones que contienen valores nulos
#IPC_Alimento = IPC_Alimento.dropna(axis=0)
IPC_Alimento = IPC_Alimento.dropna(how='all')
IPC_Alimento.head(2)

,mes,IPC general,Alimentos,No alimentos
0,1992-03-01,24.551019,20.766725,28.431095
1,1992-04-01,24.615683,20.751941,28.600619


## Cálculo inflación Alimentos

In [11]:
Infla_gral = calcular_inflacion(IPC_Alimento, 'IPC general', 'mes')
Infla_gral = Infla_gral.iloc[:, [0, 4, 5, 6]]
Infla_alim = calcular_inflacion(IPC_Alimento, 'Alimentos', 'mes')
Infla_alim = Infla_alim.iloc[:, [4, 5, 6]]
Infla_nalim = calcular_inflacion(IPC_Alimento, 'No alimentos', 'mes')
Infla_nalim = Infla_nalim.iloc[:, [4, 5, 6]]

# Concatenar los DataFrames en uno solo
infla_alimen = pd.concat([Infla_gral, Infla_alim, Infla_nalim], axis=1)
import pandas as pd
# Eliminar observaciones que contienen valores nulos
#infla_alimen = infla_alimen.dropna(axis=0)
infla_alimen = infla_alimen.dropna(how='all')
# Formato de fechas
infla_alimen.loc[:, 'mes'] = infla_alimen['mes'].dt.strftime('%b-%y')
# Redondear los decimales
infla_alimen = infla_alimen.round(10)

# Índice de precios al consumidor por productos (397 productos)

## Limpieza de datos

In [12]:
import pandas as pd
# Eliminar observaciones que contienen valores nulos
#IPC_Productos = IPC_Productos.dropna(axis=0)
IPC_Productos = IPC_Productos.dropna(how='all')
IPC_Productos.head(2)

,mes,IPC general,Pan corriente,Pan especial,Galletas,Panetón,Torta,Empanada al horno,Queques,Llauchas,...,Servicio de guarderías,Seguro relacionado con la salud,Seguro relacionado con el transporte,Servicios de asesoramiento jurídico,Gastos en pompas fúnebres y en otros servicios de funerales,Pago por fotocopias y otras reproducciones de documentos,Emisión de certificados,Roseta de inspección vehicular,Cédula de identidad CI,Servicios religiosos
0,2017-12-01,100.860240,1.944171,0.245399,0.505634,0.093038,0.202355,0.191127,0.043872,0.027014,...,0.111559,0.070172,0.089397,0.050412,0.039202,0.065664,0.040209,0.013366,0.016013,0.000761
1,2018-01-01,101.167616,1.944034,0.244824,0.509821,0.093293,0.201589,0.192480,0.043617,0.026634,...,0.113687,0.070172,0.090389,0.049549,0.039969,0.064478,0.040181,0.013366,0.016013,0.000761


## Cálculo de la inflación de los 397 productos

In [13]:
import pandas as pd

# Supongamos que tienes un DataFrame llamado IPC_Productos con 398 variables.
variables = IPC_Productos.columns.tolist()
variables = [var for var in variables if var != "mes"]

infla_resultados = pd.DataFrame()  # Dataframe para almacenar los resultados

for variable in variables:
    infla = calcular_inflacion(IPC_Productos, variable, 'mes')
    infla = infla[['mes', f'imes_{variable}', f'i12m_{variable}', f'icum_{variable}']]

    # Combina los resultados en un DataFrame más grande
    infla_resultados = pd.concat([infla_resultados, infla], axis=1)

# Ahora, infla_resultados contendrá las columnas de inflación para todas las variables
import pandas as pd
# Eliminar observaciones que contienen valores nulos
#infla_produc = infla_resultados.dropna(axis=0)
infla_produc = infla_resultados.dropna(how='all')
# Formato de fechas
# Eliminar la variable repetida
infla_produc = infla_produc.loc[:, ~infla_produc.columns.duplicated(keep='first')]
infla_produc.loc[:, 'mes'] = infla_produc['mes'].dt.strftime('%b-%y')
# Redondear los decimales
infla_produc = infla_produc.round(10)


# Índice de precios histórico

## Limpieza de datos

In [58]:
import pandas as pd
# Eliminar observaciones que contienen valores nulos
#IPC_Productos = IPC_Productos.dropna(axis=0)
IPC_Historical = IPC_Historical.dropna(how='all')
IPC_Historical.head(2)

,mes,IPC_2016,IPC_1991
0,1937-01-31,1.423620e-08,6.432980e-08
1,1937-02-28,1.458770e-08,6.591820e-08


## Cálculo de la inflación histórica base 2016 y 1992

In [73]:
Infla_2016 = calcular_inflacion(IPC_Historical, 'IPC_2016', 'mes')
Infla_2016 = Infla_2016.iloc[:, [0, 3, 4, 5]]
Infla_1991 = calcular_inflacion(IPC_Historical, 'IPC_1991', 'mes')
Infla_1991 = Infla_1991.iloc[:, [3, 4, 5]]


# Concatenar los DataFrames en uno solo
infla_histo = pd.concat([Infla_2016, Infla_1991], axis=1)
import pandas as pd
# Eliminar observaciones que contienen valores nulos
#infla_alimen = infla_alimen.dropna(axis=0)
infla_histo = infla_histo.dropna(how='all')
# Formato de fechas
infla_histo.loc[:, 'mes'] = infla_histo['mes'].dt.strftime('%b-%y')
# Redondear los decimales
infla_histo = infla_histo.round(10)

,mes,imes_IPC_2016,i12m_IPC_2016,icum_IPC_2016,imes_IPC_1991,i12m_IPC_1991,icum_IPC_1991
0,Jan-37,NaN,NaN,NaN,NaN,NaN,NaN
1,Feb-37,2.469058,NaN,NaN,2.469151,NaN,NaN
2,Mar-37,5.421691,NaN,NaN,5.421720,NaN,NaN
3,Apr-37,-6.856931,NaN,NaN,-6.857182,NaN,NaN
4,May-37,-1.840255,NaN,NaN,-1.840502,NaN,NaN
...,...,...,...,...,...,...,...
1036,May-23,0.574492,2.896795,0.571063,0.574492,2.896795,0.571063
1037,Jun-23,0.218210,2.725282,0.790519,0.218210,2.725282,0.790519
1038,Jul-23,0.367536,2.700872,1.160961,0.367536,2.700872,1.160961
1039,Aug-23,0.386201,3.058496,1.551645,0.386201,3.058496,1.551645


# Exportar resultados a un archivo excel

In [14]:
# Supongamos que tienes un DataFrame llamado df y deseas guardarlo en un archivo llamado 'mi_dataframe.xlsx' en tu carpeta de Google Drive.
ruta_archivo = '/content/drive/MyDrive/Sector Precios/Resultados Sector Precios.xlsx'
infla_depto.to_excel(ruta_archivo, sheet_name="inflación departamento", index=False)  # El argumento index=False evita que se guarde el índice en el archivo Excel.

# Carga el archivo Excel existente.
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    infla_divis.to_excel(writer, sheet_name="inflación divisiones", index=False)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    infla_alimen.to_excel(writer, sheet_name="inflación alimentos", index=False)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    infla_produc.to_excel(writer, sheet_name="inflación productos", index=False)


# Organización dataframes (Por inflación mensual, a doce meses, y acumulada)



In [15]:
#===============================================#
# Seleccionamos las lista de todas las variables de departamentos.
variables_depto = infla_depto.columns.tolist()
# Creamos un dataframe con todas las inflaciones mensual/12meses/acumulada por departamento
depto_imes = infla_depto[ ['mes'] + [variable for variable in infla_depto.columns if variable.startswith('imes_')] ]
depto_imes = depto_imes.rename(columns=lambda x: x.replace('imes_', '') if x.startswith('imes_') else x) # Retornamos nombre original
depto_imes.set_index('mes', inplace=True)
depto_imes = depto_imes.dropna(how='all')
depto_i12m = infla_depto[ ['mes'] + [variable for variable in infla_depto.columns if variable.startswith('i12m_')] ]
depto_i12m = depto_i12m.rename(columns=lambda x: x.replace('i12m_', '') if x.startswith('i12m_') else x) # Retornamos nombre original
depto_i12m.set_index('mes', inplace=True)
depto_i12m = depto_i12m.dropna(how='all')
depto_icum = infla_depto[ ['mes'] + [variable for variable in infla_depto.columns if variable.startswith('icum_')] ]
depto_icum = depto_icum.rename(columns=lambda x: x.replace('icum_', '') if x.startswith('icum_') else x) # Retornamos nombre original
depto_icum.set_index('mes', inplace=True)
depto_icum = depto_icum.dropna(how='all')
#===============================================#
# Seleccionamos las lista de todas las variables de divisiones.
variables_divis = infla_divis.columns.tolist()
# Creamos un dataframe con todas las inflaciones mensual/12meses/acumulada por departamento
divis_imes = infla_divis[ ['mes'] + [variable for variable in infla_divis.columns if variable.startswith('imes_')] ]
divis_imes = divis_imes.rename(columns=lambda x: x.replace('imes_', '') if x.startswith('imes_') else x) # Retornamos nombre original
divis_imes.set_index('mes', inplace=True)
divis_imes = divis_imes.dropna(how='all')
divis_i12m = infla_divis[ ['mes'] + [variable for variable in infla_divis.columns if variable.startswith('i12m_')] ]
divis_i12m = divis_i12m.rename(columns=lambda x: x.replace('i12m_', '') if x.startswith('i12m_') else x) # Retornamos nombre original
divis_i12m.set_index('mes', inplace=True)
divis_i12m = divis_i12m.dropna(how='all')
divis_icum = infla_divis[ ['mes'] + [variable for variable in infla_divis.columns if variable.startswith('icum_')] ]
divis_icum = divis_icum.rename(columns=lambda x: x.replace('icum_', '') if x.startswith('icum_') else x) # Retornamos nombre original
divis_icum.set_index('mes', inplace=True)
divis_icum = divis_icum.dropna(how='all')
#===============================================#
# Seleccionamos las lista de todas las variables de productos.
variables_produc = infla_produc.columns.tolist()
# Creamos un dataframe con todas las inflaciones mensual/12meses/acumulada por departamento
produc_imes = infla_produc[ ['mes'] + [variable for variable in infla_produc.columns if variable.startswith('imes_')] ]
produc_imes = produc_imes.rename(columns=lambda x: x.replace('imes_', '') if x.startswith('imes_') else x) # Retornamos nombre original
produc_imes.set_index('mes', inplace=True)
produc_imes = produc_imes.dropna(how='all')
produc_i12m = infla_produc[ ['mes'] + [variable for variable in infla_produc.columns if variable.startswith('i12m_')] ]
produc_i12m = produc_i12m.rename(columns=lambda x: x.replace('i12m_', '') if x.startswith('i12m_') else x) # Retornamos nombre original
produc_i12m.set_index('mes', inplace=True)
produc_i12m = produc_i12m.dropna(how='all')
produc_icum = infla_produc[ ['mes'] + [variable for variable in infla_produc.columns if variable.startswith('icum_')] ]
produc_icum = produc_icum.rename(columns=lambda x: x.replace('icum_', '') if x.startswith('icum_') else x) # Retornamos nombre original
produc_icum.set_index('mes', inplace=True)
produc_icum = produc_icum.dropna(how='all')
#===============================================#
# Seleccionamos las lista de todas las variables de alimentos.
variables_alimen = infla_alimen.columns.tolist()
# Creamos un dataframe con todas las inflaciones mensual/12meses/acumulada por departamento
alimen_imes = infla_alimen[ ['mes'] + [variable for variable in infla_alimen.columns if variable.startswith('imes_')] ]
alimen_imes = alimen_imes.rename(columns=lambda x: x.replace('imes_', '') if x.startswith('imes_') else x) # Retornamos nombre original
alimen_imes.set_index('mes', inplace=True)
alimen_imes = alimen_imes.dropna(how='all')
alimen_i12m = infla_alimen[ ['mes'] + [variable for variable in infla_alimen.columns if variable.startswith('i12m_')] ]
alimen_i12m = alimen_i12m.rename(columns=lambda x: x.replace('i12m_', '') if x.startswith('i12m_') else x) # Retornamos nombre original
alimen_i12m.set_index('mes', inplace=True)
alimen_i12m = alimen_i12m.dropna(how='all')
alimen_icum = infla_alimen[ ['mes'] + [variable for variable in infla_alimen.columns if variable.startswith('icum_')] ]
alimen_icum = alimen_icum.rename(columns=lambda x: x.replace('icum_', '') if x.startswith('icum_') else x) # Retornamos nombre original
alimen_icum.set_index('mes', inplace=True)
alimen_icum = alimen_icum.dropna(how='all')
#===============================================#
#===============================================#
# Seleccionamos las lista de todas las variables de inflación histórica.
variables_alimen = infla_alimen.columns.tolist()
# Creamos un dataframe con todas las inflaciones mensual/12meses/acumulada por departamento
alimen_imes = infla_alimen[ ['mes'] + [variable for variable in infla_alimen.columns if variable.startswith('imes_')] ]
alimen_imes = alimen_imes.rename(columns=lambda x: x.replace('imes_', '') if x.startswith('imes_') else x) # Retornamos nombre original
alimen_imes.set_index('mes', inplace=True)
alimen_imes = alimen_imes.dropna(how='all')
alimen_i12m = infla_alimen[ ['mes'] + [variable for variable in infla_alimen.columns if variable.startswith('i12m_')] ]
alimen_i12m = alimen_i12m.rename(columns=lambda x: x.replace('i12m_', '') if x.startswith('i12m_') else x) # Retornamos nombre original
alimen_i12m.set_index('mes', inplace=True)
alimen_i12m = alimen_i12m.dropna(how='all')
alimen_icum = infla_alimen[ ['mes'] + [variable for variable in infla_alimen.columns if variable.startswith('icum_')] ]
alimen_icum = alimen_icum.rename(columns=lambda x: x.replace('icum_', '') if x.startswith('icum_') else x) # Retornamos nombre original
alimen_icum.set_index('mes', inplace=True)
alimen_icum = alimen_icum.dropna(how='all')
#===============================================#

# Cálculo de la incidencia de la inflación (_mensual_, _a doce meses_ y _acumulada_)

## Incidencia por departamento


In [16]:
# Utilizamos los dataframe de ponderadores y aplicamos a la inflación mensual; a doce meses; y acumulada
# Para la inflación mensual
depto_imes_inc = pd.DataFrame()
depto_imes_inc['Bolivia'] = depto_imes['Bolivia']       *w0_depto.iloc[0, 0] / 100
depto_imes_inc['Sucre'] = depto_imes['Sucre']           *w0_depto.iloc[0, 1] / 100
depto_imes_inc['La Paz'] = depto_imes['La Paz']         *w0_depto.iloc[0, 2] / 100
depto_imes_inc['Cochabamba'] = depto_imes['Cochabamba'] *w0_depto.iloc[0, 3] / 100
depto_imes_inc['Oruro'] = depto_imes['Oruro']           *w0_depto.iloc[0, 4] / 100
depto_imes_inc['Potosí'] = depto_imes['Potosí']         *w0_depto.iloc[0, 5] / 100
depto_imes_inc['Tarija'] = depto_imes['Tarija']         *w0_depto.iloc[0, 6] / 100
depto_imes_inc['Santa Cruz'] = depto_imes['Santa Cruz'] *w0_depto.iloc[0, 7] / 100
depto_imes_inc['Beni'] = depto_imes['Beni']             *w0_depto.iloc[0, 8] / 100
depto_imes_inc['Pando'] = depto_imes['Pando']           *w0_depto.iloc[0, 9] / 100
# Para la inflación a doce meses
depto_i12m_inc = pd.DataFrame()
depto_i12m_inc['Bolivia'] = depto_i12m['Bolivia']       *w0_depto.iloc[0, 0] / 100
depto_i12m_inc['Sucre'] = depto_i12m['Sucre']           *w0_depto.iloc[0, 1] / 100
depto_i12m_inc['La Paz'] = depto_i12m['La Paz']         *w0_depto.iloc[0, 2] / 100
depto_i12m_inc['Cochabamba'] = depto_i12m['Cochabamba'] *w0_depto.iloc[0, 3] / 100
depto_i12m_inc['Oruro'] = depto_i12m['Oruro']           *w0_depto.iloc[0, 4] / 100
depto_i12m_inc['Potosí'] = depto_i12m['Potosí']         *w0_depto.iloc[0, 5] / 100
depto_i12m_inc['Tarija'] = depto_i12m['Tarija']         *w0_depto.iloc[0, 6] / 100
depto_i12m_inc['Santa Cruz'] = depto_i12m['Santa Cruz'] *w0_depto.iloc[0, 7] / 100
depto_i12m_inc['Beni'] = depto_i12m['Beni']             *w0_depto.iloc[0, 8] / 100
depto_i12m_inc['Pando'] = depto_i12m['Pando']           *w0_depto.iloc[0, 9] / 100
# Para la inflación acumulada
depto_icum_inc = pd.DataFrame()
depto_icum_inc['Bolivia'] = depto_icum['Bolivia']       *w0_depto.iloc[0, 0] / 100
depto_icum_inc['Sucre'] = depto_icum['Sucre']           *w0_depto.iloc[0, 1] / 100
depto_icum_inc['La Paz'] = depto_icum['La Paz']         *w0_depto.iloc[0, 2] / 100
depto_icum_inc['Cochabamba'] = depto_icum['Cochabamba'] *w0_depto.iloc[0, 3] / 100
depto_icum_inc['Oruro'] = depto_icum['Oruro']           *w0_depto.iloc[0, 4] / 100
depto_icum_inc['Potosí'] = depto_icum['Potosí']         *w0_depto.iloc[0, 5] / 100
depto_icum_inc['Tarija'] = depto_icum['Tarija']         *w0_depto.iloc[0, 6] / 100
depto_icum_inc['Santa Cruz'] = depto_icum['Santa Cruz'] *w0_depto.iloc[0, 7] / 100
depto_icum_inc['Beni'] = depto_icum['Beni']             *w0_depto.iloc[0, 8] / 100
depto_icum_inc['Pando'] = depto_icum['Pando']           *w0_depto.iloc[0, 9] / 100

## Incidencia por categoria de alimentos y no alimentos

In [17]:
# Utilizamos los dataframe de ponderadores y aplicamos a la inflación mensual; a doce meses; y acumulada
# Para la inflación mensual
alimen_imes_inc = pd.DataFrame()
alimen_imes_inc['IPC general'] = alimen_imes['IPC general']   *w0_alimen.iloc[0, 0] / 100
alimen_imes_inc['Alimentos'] = alimen_imes['Alimentos']       *w0_alimen.iloc[0, 1] / 100
alimen_imes_inc['No alimentos'] = alimen_imes['No alimentos'] *w0_alimen.iloc[0, 2] / 100
# Para la inflación a doce meses
alimen_i12m_inc = pd.DataFrame()
alimen_i12m_inc['IPC general'] = alimen_i12m['IPC general']   *w0_alimen.iloc[0, 0] / 100
alimen_i12m_inc['Alimentos'] = alimen_i12m['Alimentos']       *w0_alimen.iloc[0, 1] / 100
alimen_i12m_inc['No alimentos'] = alimen_i12m['No alimentos'] *w0_alimen.iloc[0, 2] / 100
# Para la inflación acumulada
alimen_icum_inc = pd.DataFrame()
alimen_icum_inc['IPC general'] = alimen_icum['IPC general']   *w0_alimen.iloc[0, 0] / 100
alimen_icum_inc['Alimentos'] = alimen_icum['Alimentos']       *w0_alimen.iloc[0, 1] / 100
alimen_icum_inc['No alimentos'] = alimen_icum['No alimentos'] *w0_alimen.iloc[0, 2] / 100


## Incidencia por divisiones

In [18]:
# Para la inflación mensual
divis_imes_inc = pd.DataFrame()
divis_imes_inc['Bolivia'] = divis_imes['Bolivia']                                                               *w0_divis.iloc[0, 0] / 100
divis_imes_inc['Alimentos y bebidas no alcohólicas'] = divis_imes['Alimentos y bebidas no alcohólicas']         *w0_divis.iloc[0, 1] / 100
divis_imes_inc['Bebidas alcohólicas y tabaco'] = divis_imes['Bebidas alcohólicas y tabaco']                     *w0_divis.iloc[0, 2] / 100
divis_imes_inc['Prendas de vestir y calzado'] = divis_imes['Prendas de vestir y calzado']                     *w0_divis.iloc[0, 3] / 100
divis_imes_inc['Vivienda y servicios básicos'] = divis_imes['Vivienda y servicios básicos']                     *w0_divis.iloc[0, 4] / 100
divis_imes_inc['Muebles, bienes y servicios domésticos'] = divis_imes['Muebles, bienes y servicios domésticos'] *w0_divis.iloc[0, 5] / 100
divis_imes_inc['Salud'] = divis_imes['Salud']                                                                   *w0_divis.iloc[0, 6] / 100
divis_imes_inc['Transporte'] = divis_imes['Transporte']                                                         *w0_divis.iloc[0, 7] / 100
divis_imes_inc['Comunicaciones'] = divis_imes['Comunicaciones']                                                 *w0_divis.iloc[0, 8] / 100
divis_imes_inc['Recreación y cultura'] = divis_imes['Recreación y cultura']                                     *w0_divis.iloc[0, 9] / 100
divis_imes_inc['Educación'] = divis_imes['Educación']                                                           *w0_divis.iloc[0, 10] / 100
divis_imes_inc['Alimentos y bebidas consumidos fuera del hogar'] = divis_imes['Alimentos y bebidas consumidos fuera del hogar'] *w0_divis.iloc[0, 11] / 100
divis_imes_inc['Bienes y servicios diversos'] = divis_imes['Bienes y servicios diversos']                       *w0_divis.iloc[0, 12] / 100
# Para la inflación a doce meses
divis_i12m_inc = pd.DataFrame()
divis_i12m_inc['Bolivia'] = divis_i12m['Bolivia']                                                               *w0_divis.iloc[0, 0] / 100
divis_i12m_inc['Alimentos y bebidas no alcohólicas'] = divis_i12m['Alimentos y bebidas no alcohólicas']         *w0_divis.iloc[0, 1] / 100
divis_i12m_inc['Bebidas alcohólicas y tabaco'] = divis_i12m['Bebidas alcohólicas y tabaco']                     *w0_divis.iloc[0, 2] / 100
divis_i12m_inc['Prendas de vestir y calzado'] = divis_i12m['Prendas de vestir y calzado']                     *w0_divis.iloc[0, 3] / 100
divis_i12m_inc['Vivienda y servicios básicos'] = divis_i12m['Vivienda y servicios básicos']                     *w0_divis.iloc[0, 4] / 100
divis_i12m_inc['Muebles, bienes y servicios domésticos'] = divis_i12m['Muebles, bienes y servicios domésticos'] *w0_divis.iloc[0, 5] / 100
divis_i12m_inc['Salud'] = divis_i12m['Salud']                                                                   *w0_divis.iloc[0, 6] / 100
divis_i12m_inc['Transporte'] = divis_i12m['Transporte']                                                         *w0_divis.iloc[0, 7] / 100
divis_i12m_inc['Comunicaciones'] = divis_i12m['Comunicaciones']                                                 *w0_divis.iloc[0, 8] / 100
divis_i12m_inc['Recreación y cultura'] = divis_i12m['Recreación y cultura']                                     *w0_divis.iloc[0, 9] / 100
divis_i12m_inc['Educación'] = divis_i12m['Educación']                                                           *w0_divis.iloc[0, 10] / 100
divis_i12m_inc['Alimentos y bebidas consumidos fuera del hogar'] = divis_i12m['Alimentos y bebidas consumidos fuera del hogar'] *w0_divis.iloc[0, 11] / 100
divis_i12m_inc['Bienes y servicios diversos'] = divis_i12m['Bienes y servicios diversos']                       *w0_divis.iloc[0, 12] / 100
# Para la inflación acumulada
divis_icum_inc = pd.DataFrame()
divis_icum_inc['Bolivia'] = divis_icum['Bolivia']                                                               *w0_divis.iloc[0, 0] / 100
divis_icum_inc['Alimentos y bebidas no alcohólicas'] = divis_icum['Alimentos y bebidas no alcohólicas']         *w0_divis.iloc[0, 1] / 100
divis_icum_inc['Bebidas alcohólicas y tabaco'] = divis_icum['Bebidas alcohólicas y tabaco']                     *w0_divis.iloc[0, 2] / 100
divis_icum_inc['Prendas de vestir y calzado'] = divis_icum['Prendas de vestir y calzado']                     *w0_divis.iloc[0, 3] / 100
divis_icum_inc['Vivienda y servicios básicos'] = divis_icum['Vivienda y servicios básicos']                     *w0_divis.iloc[0, 4] / 100
divis_icum_inc['Muebles, bienes y servicios domésticos'] = divis_icum['Muebles, bienes y servicios domésticos'] *w0_divis.iloc[0, 5] / 100
divis_icum_inc['Salud'] = divis_icum['Salud']                                                                   *w0_divis.iloc[0, 6] / 100
divis_icum_inc['Transporte'] = divis_icum['Transporte']                                                         *w0_divis.iloc[0, 7] / 100
divis_icum_inc['Comunicaciones'] = divis_icum['Comunicaciones']                                                 *w0_divis.iloc[0, 8] / 100
divis_icum_inc['Recreación y cultura'] = divis_icum['Recreación y cultura']                                     *w0_divis.iloc[0, 9] / 100
divis_icum_inc['Educación'] = divis_icum['Educación']                                                           *w0_divis.iloc[0, 10] / 100
divis_icum_inc['Alimentos y bebidas consumidos fuera del hogar'] = divis_icum['Alimentos y bebidas consumidos fuera del hogar'] *w0_divis.iloc[0, 11] / 100
divis_icum_inc['Bienes y servicios diversos'] = divis_icum['Bienes y servicios diversos']                       *w0_divis.iloc[0, 12] / 100


## Exportar todos los dataframes de incidencia _mensual_, _a doce meses_ y _acumulada_ por categoria
- Departamento
- Divisiones
- Alimentos

In [19]:
# Incidencias
# Incidencia departamental
ruta_archivo = '/content/drive/MyDrive/Sector Precios/Resultados Incidencia Desagregado.xlsx'
depto_imes_inc.to_excel(ruta_archivo, sheet_name="inc. mens. depto", index=True)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    depto_i12m_inc.to_excel(writer, sheet_name="inc. a12m. depto", index=True)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    depto_icum_inc.to_excel(writer, sheet_name="inc. acum. depto", index=True)
# Incidencia alimentos y no alimentos
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    alimen_imes_inc.to_excel(writer, sheet_name="inc. mens. alim.", index=True)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    alimen_i12m_inc.to_excel(writer, sheet_name="inc. a12m. alim.", index=True)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    alimen_icum_inc.to_excel(writer, sheet_name="inc. acum. alim.", index=True)
    # Incidencia divisiones
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    divis_imes_inc.to_excel(writer, sheet_name="inc. mens. división", index=True)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    divis_i12m_inc.to_excel(writer, sheet_name="inc. a12m. división", index=True)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    divis_icum_inc.to_excel(writer, sheet_name="inc. acum. división", index=True)

## Exportar todos los dataframes de _inflación mensual_, _a doce meses_ y _acumulada_ por categoria
- Departamento
- Divisiones
- Alimentos
- Productos

In [20]:
# Supongamos que tienes un DataFrame llamado df y deseas guardarlo en un archivo llamado 'mi_dataframe.xlsx' en tu carpeta de Google Drive.
# Inflación departamental
ruta_archivo = '/content/drive/MyDrive/Sector Precios/Resultados Precios Desagregado.xlsx'
depto_imes.to_excel(ruta_archivo, sheet_name="inf. mens. depto", index=True)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    depto_i12m.to_excel(writer, sheet_name="inf. a12m. depto", index=True)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    depto_icum.to_excel(writer, sheet_name="inf. acum. depto", index=True)
# Inflación por Divisiones
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    divis_imes.to_excel(writer, sheet_name="inf. mens. división", index=True)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    divis_i12m.to_excel(writer, sheet_name="inf. a12m. división", index=True)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    divis_icum.to_excel(writer, sheet_name="inf. acum. división", index=True)
# Inflación por Productos (397)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    produc_imes.to_excel(writer, sheet_name="inf. mens. producto", index=True)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    produc_i12m.to_excel(writer, sheet_name="inf. a12m. producto", index=True)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    produc_icum.to_excel(writer, sheet_name="inf. acum. producto", index=True)
# Inflación por Alimentos
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    alimen_imes.to_excel(writer, sheet_name="inf. mens. alim.", index=True)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    alimen_i12m.to_excel(writer, sheet_name="inf. a12m. alim.", index=True)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    alimen_icum.to_excel(writer, sheet_name="inf. acum. alim.", index=True)

# Función High-to-Low para datos de frecuencia Mensual a Anual (Criterio Last - Average)

In [21]:
depto_icum

,Bolivia,Sucre,La Paz,Cochabamba,Oruro,Potosí,Tarija,Santa Cruz,Beni,Pando
mes,,,,,,,,,,
Jan-94,0.831143,NaN,0.626764,0.992367,NaN,NaN,NaN,1.083888,NaN,NaN
Feb-94,1.380217,NaN,1.507842,1.640425,NaN,NaN,NaN,1.160431,NaN,NaN
Mar-94,1.294237,NaN,1.289606,1.855445,NaN,NaN,NaN,0.987268,NaN,NaN
Apr-94,1.789721,NaN,1.031197,4.050229,NaN,NaN,NaN,1.488398,NaN,NaN
May-94,2.411207,NaN,1.289172,5.333402,NaN,NaN,NaN,2.274409,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
May-23,0.571063,0.205679,0.886875,0.060877,0.579615,-0.138078,-0.833202,0.755358,0.429766,0.146397
Jun-23,0.790519,0.543418,1.071079,0.209135,0.917164,0.020206,-0.638980,1.023831,0.471664,0.513782
Jul-23,1.160961,1.473701,1.475524,0.764633,1.397877,0.982814,0.256141,1.126014,0.448224,0.594219


In [27]:
depto_icum_anual = depto_icum

,Bolivia,Sucre,La Paz,Cochabamba,Oruro,Potosí,Tarija,Santa Cruz,Beni,Pando
mes,,,,,,,,,,
Jan-94,0.831143,NaN,0.626764,0.992367,NaN,NaN,NaN,1.083888,NaN,NaN
Feb-94,1.380217,NaN,1.507842,1.640425,NaN,NaN,NaN,1.160431,NaN,NaN
Mar-94,1.294237,NaN,1.289606,1.855445,NaN,NaN,NaN,0.987268,NaN,NaN
Apr-94,1.789721,NaN,1.031197,4.050229,NaN,NaN,NaN,1.488398,NaN,NaN
May-94,2.411207,NaN,1.289172,5.333402,NaN,NaN,NaN,2.274409,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
May-23,0.571063,0.205679,0.886875,0.060877,0.579615,-0.138078,-0.833202,0.755358,0.429766,0.146397
Jun-23,0.790519,0.543418,1.071079,0.209135,0.917164,0.020206,-0.638980,1.023831,0.471664,0.513782
Jul-23,1.160961,1.473701,1.475524,0.764633,1.397877,0.982814,0.256141,1.126014,0.448224,0.594219


In [28]:
# Convertir el índice 'mensual month-year Jan-09' a formato de fecha
depto_icum_anual.index = pd.to_datetime(depto_icum_anual.index, format='%b-%y')

# Crear la columna 'yearly' con el año de cada fila
depto_icum_anual['yearly'] = depto_icum_anual.index.year

# Agrupar por año, y tomar el último valor de cada año para cada variable
depto_icum_last_year_value = depto_icum_anual.groupby(depto_icum_anual.index.year).last()
# Eliminar la variable auxiliar "yearly"
#depto_icum_last_year_value = depto_icum_last_year_value.drop(columns=['yearly'])

depto_icum_last_year_value

,Bolivia,Sucre,La Paz,Cochabamba,Oruro,Potosí,Tarija,Santa Cruz,Beni,Pando,yearly
mes,,,,,,,,,,,
1994,8.521696,NaN,8.247584,11.389796,NaN,NaN,NaN,6.675599,NaN,NaN,1994
1995,12.577858,NaN,11.010992,13.485024,NaN,NaN,NaN,14.335475,NaN,NaN,1995
1996,7.949922,NaN,7.354512,8.612764,NaN,NaN,NaN,8.438004,NaN,NaN,1996
1997,6.730938,NaN,6.185080,5.840005,NaN,NaN,NaN,8.820858,NaN,NaN,1997
1998,4.392166,NaN,3.782233,3.899333,NaN,NaN,NaN,5.582120,NaN,NaN,1998
1999,3.133487,NaN,2.369236,3.232848,NaN,NaN,NaN,4.053967,NaN,NaN,1999
2000,3.411911,NaN,3.197058,3.667819,NaN,NaN,NaN,3.499551,NaN,NaN,2000
2001,0.924897,NaN,0.588998,1.036298,NaN,NaN,NaN,1.319160,NaN,NaN,2001
2002,2.445905,NaN,1.696689,3.022006,NaN,NaN,NaN,2.875502,NaN,NaN,2002


# Gráficos de las variables para reportes

# Configuración Gráficas Colores

In [29]:
from pandas._libs.tslibs.period import Period
df2 = depto_icum_last_year_value.query('yearly >= 2009').filter(items=['yearly', 'Bolivia','Sucre','La Paz',	'Cochabamba',	'Oruro',	'Potosí',	'Tarija',	'Santa Cruz',	'Beni',	'Pando']).reset_index()
df2['mescp'] = 1
df2.loc[df2['yearly']<2006, 'mescp'] = 0
df2.loc[df2['yearly']==2020, 'mescp'] = 0
# Colores anteriores
#df2['colors'] = '#d1ceeb'
#df2['colors'] = '#DC143C'
#df2['colors'] = '#C90016'
df2['colors'] = '#AF4035'
#df2.loc[df2['mescp'] > 0, 'colors'] = '#090382'
df2.loc[df2['mescp'] > 0, 'colors'] = '#002E63'
df2.tail(20)

,mes,yearly,Bolivia,Sucre,La Paz,Cochabamba,Oruro,Potosí,Tarija,Santa Cruz,Beni,Pando,mescp,colors
0,2009,2009,0.263903,-0.161840,-0.245054,0.540079,-1.281070,-2.347963,0.254283,0.958176,-0.550408,-4.260042,1,#002E63
1,2010,2010,7.181850,7.786219,6.976608,7.370431,7.654490,7.346263,7.106646,7.468945,3.674872,5.972747,1,#002E63
2,2011,2011,6.903542,6.353547,7.666180,6.342859,7.585848,7.815096,9.356695,6.420273,5.253332,9.501815,1,#002E63
3,2012,2012,4.540112,3.409843,5.029448,5.984080,3.520780,4.043065,4.053635,3.963048,1.519713,1.979937,1,#002E63
4,2013,2013,6.479683,8.400975,7.214700,6.164930,9.028860,7.546586,8.310275,5.809019,3.399959,1.936533,1,#002E63
5,2014,2014,5.194248,4.536910,4.034262,7.442834,3.865096,4.067775,9.084704,4.761118,5.553369,2.029914,1,#002E63
6,2015,2015,2.953146,2.960001,4.603151,4.082901,1.792393,1.221698,1.743358,1.412362,2.999359,-0.539200,1,#002E63
7,2016,2016,4.002939,4.899343,4.600009,4.869843,3.494710,5.696221,7.512187,2.747716,1.452062,0.826266,1,#002E63
8,2017,2017,2.714668,2.457785,1.975067,4.005971,1.720920,1.511708,11.731364,1.863317,1.837252,-1.824674,1,#002E63
9,2018,2018,1.507067,1.494139,1.874310,1.598757,1.410021,0.913092,1.658194,1.178464,0.544421,2.352107,1,#002E63


In [30]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(x=df2.yearly, y=df2["Bolivia"], name="Depto_inflation_bol",
                marker_color=df2.colors, marker_line=dict(width=0.5, color='#000000') ))
fig.update_layout(font_family = 'Arial', font_color="#000000", plot_bgcolor='white', separators=',',
                  title_text='Bolivia: Inflación, 2009-2023',
                  title_font = dict(color='#000000', size=30),
                  xaxis=dict(title='Año', titlefont_size=20, tickfont_size=20),
                  yaxis=dict(title='En porcentaje', titlefont_size=20, tickfont_size=18, linecolor="#000000", ticks='outside', nticks=10, zeroline=True, zerolinewidth=1.5, zerolinecolor='#d62f15'),
                  hoverlabel=dict(font_size=20,font_family="Arial"),
                  hovermode="x"
                  )

In [31]:
import plotly.graph_objects as go

# Suponiendo que tienes una columna 'values' en tu DataFrame df2 con los valores que deseas mostrar
fig = go.Figure()

# Añade la columna 'text' con los valores que quieres mostrar en cada barra
df2['text'] = df2["Bolivia"].round(2)  # Ajusta el redondeo según lo necesites

fig.add_trace(go.Bar(
    x=df2.yearly,
    y=df2["Bolivia"],
    name="Depto_inflation_bol",
    marker_color=df2.colors,
    marker_line=dict(width=0.5, color='#000000'),
    text=df2['text'],  # Añade la propiedad 'text' aquí
    textposition='outside',  # Esto determina la posición del texto en las barras
    textfont=dict(
        family="Arial",  # Puedes cambiar a la fuente que prefieras
        size=16,  # Tamaño del texto
        color="black",  # Puedes cambiar a un color más oscuro si lo prefieres
    )
))

# Resto del código para actualizar el layout
fig.update_layout(
    font_family='Arial',
    font_color="#000000",
    plot_bgcolor='white',
    separators=',',
    title_text='Bolivia: Inflación, 2009-2023',
    title_font=dict(color='#000000', size=30),
    xaxis=dict(title='Año', titlefont_size=20, tickfont_size=20),
    yaxis=dict(
        title='En porcentaje',
        titlefont_size=20,
        tickfont_size=18,
        linecolor="#000000",
        ticks='outside',
        nticks=10,
        zeroline=True,
        zerolinewidth=1.5,
        zerolinecolor='#d62f15'
    ),
    hoverlabel=dict(font_size=20, font_family="Arial"),
    hovermode="x"
)

fig.show()



In [32]:
import plotly.graph_objects as go

# Suponiendo que tienes una columna 'values' en tu DataFrame df2 con los valores que deseas mostrar
fig = go.Figure()

# Añade la columna 'text' con los valores que quieres mostrar en cada barra
df2['text'] = df2["Bolivia"].round(2)  # Ajusta el redondeo según lo necesites

bar_trace = go.Bar(
    x=df2.yearly,
    y=df2["Bolivia"],
    name="Depto_inflation_bol",
    marker_color=df2.colors,
    marker_line=dict(width=0.5, color='#000000'),
    text=df2['text'],  # Añade la propiedad 'text' aquí
    textposition='outside',  # Esto determina la posición del texto en las barras
    textfont=dict(
        family="Arial",  # Puedes cambiar a la fuente que prefieras
        size=16,  # Tamaño del texto
        color="black",  # Puedes cambiar a un color más oscuro si lo prefieres
    )
)

fig.add_trace(bar_trace)

# Agregar el texto "A septiembre" en la última barra
last_value = df2["Bolivia"].iloc[-1]  # Obtener el último valor de la columna "Bolivia"
fig.add_annotation(
    x=df2.yearly.iloc[-1],  # Posición x de la última barra
    y=last_value,  # Posición y del último valor
    text="A septiembre",  # Texto a mostrar
    #showarrow=True,  # Mostrar una flecha hacia el texto (opcional)
    showarrow=False,  # Mostrar una flecha hacia el texto (opcional)
        font=dict(
        family="Arial",
        size=15,
        color="black"
    ),
    arrowhead=7,  # Tipo de flecha (opcional)
    #ax=0,  # Posición x de inicio de la flecha (opcional)
    #ay=-140  # Posición y de inicio de la flecha (opcional)
    yshift=-80  # Mover el texto hacia abajo para que esté por debajo de la barra
)

# Resto del código para actualizar el layout
fig.update_layout(
    font_family='Arial',
    font_color="#000000",
    plot_bgcolor='white',
    separators=',',
    title_text='Bolivia: Inflación, 2009-2023',
    title_font=dict(color='#000000', size=30),
    xaxis=dict(title='Año', titlefont_size=20, tickfont_size=20),
    yaxis=dict(
        title='En porcentaje',
        titlefont_size=20,
        tickfont_size=18,
        linecolor="#000000",
        ticks='outside',
        nticks=10,
        zeroline=True,
        zerolinewidth=1.5,
        zerolinecolor='#d62f15'
    ),
    hoverlabel=dict(font_size=20, font_family="Arial"),
    hovermode="x"
)

fig.show()


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(x=df2.yearly, y=df2["Sucre"], name="Depto_inflation_sucr",
                marker_color=df2.colors, marker_line=dict(width=0.5, color='#000000') ))
fig.update_layout(font_family = 'Arial', font_color="#000000", plot_bgcolor='white', separators=',',
                  title_text='Sucre: Inflación, 2009-2023',
                  title_font = dict(color='#000000', size=30),
                  xaxis=dict(title='Año', titlefont_size=20, tickfont_size=20),
                  yaxis=dict(title='En porcentaje', titlefont_size=20, tickfont_size=18, linecolor="#000000", ticks='outside', nticks=10, zeroline=True, zerolinewidth=1.5, zerolinecolor='#d62f15'),
                  hoverlabel=dict(font_size=20,font_family="Arial"),
                  hovermode="x"
                  )

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df2.yearly, y=df2["La Paz"], name="Depto_inflation_lpz",
                marker_color=df2.colors, marker_line=dict(width=0.5, color='#000000') ))
fig.update_layout(font_family = 'Arial', font_color="#000000", plot_bgcolor='white', separators=',',
                  title_text='La Paz: Inflación, 2009-2023',
                  title_font = dict(color='#000000', size=30),
                  xaxis=dict(title='Año', titlefont_size=20, tickfont_size=20),
                  yaxis=dict(title='En porcentaje', titlefont_size=20, tickfont_size=18, linecolor="#000000", ticks='outside', nticks=10, zeroline=True, zerolinewidth=1.5, zerolinecolor='#d62f15'),
                  hoverlabel=dict(font_size=20,font_family="Arial"),
                  hovermode="x"
                  )

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df2.yearly, y=df2["Cochabamba"], name="Depto_inflation_cbba",
                marker_color=df2.colors, marker_line=dict(width=0.5, color='#000000') ))
fig.update_layout(font_family = 'Arial', font_color="#000000", plot_bgcolor='white', separators=',',
                  title_text='Cochabamba: Inflación, 2009-2023',
                  title_font = dict(color='#000000', size=30),
                  xaxis=dict(title='Año', titlefont_size=20, tickfont_size=20),
                  yaxis=dict(title='En porcentaje', titlefont_size=20, tickfont_size=18, linecolor="#000000", ticks='outside', nticks=10, zeroline=True, zerolinewidth=1.5, zerolinecolor='#d62f15'),
                  hoverlabel=dict(font_size=20,font_family="Arial"),
                  hovermode="x"
                  )

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df2.yearly, y=df2["Oruro"], name="Depto_inflation_orur",
                marker_color=df2.colors, marker_line=dict(width=0.5, color='#000000') ))
fig.update_layout(font_family = 'Arial', font_color="#000000", plot_bgcolor='white', separators=',',
                  title_text='Oruro: Inflación, 2009-2023',
                  title_font = dict(color='#000000', size=30),
                  xaxis=dict(title='Año', titlefont_size=20, tickfont_size=20),
                  yaxis=dict(title='En porcentaje', titlefont_size=20, tickfont_size=18, linecolor="#000000", ticks='outside', nticks=10, zeroline=True, zerolinewidth=1.5, zerolinecolor='#d62f15'),
                  hoverlabel=dict(font_size=20,font_family="Arial"),
                  hovermode="x"
                  )

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df2.yearly, y=df2["Potosí"], name="Depto_inflation_ptsi",
                marker_color=df2.colors, marker_line=dict(width=0.5, color='#000000') ))
fig.update_layout(font_family = 'Arial', font_color="#000000", plot_bgcolor='white', separators=',',
                  title_text='Potosí: Inflación, 2009-2023',
                  title_font = dict(color='#000000', size=30),
                  xaxis=dict(title='Año', titlefont_size=20, tickfont_size=20),
                  yaxis=dict(title='En porcentaje', titlefont_size=20, tickfont_size=18, linecolor="#000000", ticks='outside', nticks=10, zeroline=True, zerolinewidth=1.5, zerolinecolor='#d62f15'),
                  hoverlabel=dict(font_size=20,font_family="Arial"),
                  hovermode="x"
                  )

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df2.yearly, y=df2["Tarija"], name="Depto_inflation_tarj",
                marker_color=df2.colors, marker_line=dict(width=0.5, color='#000000') ))
fig.update_layout(font_family = 'Arial', font_color="#000000", plot_bgcolor='white', separators=',',
                  title_text='Tarija: Inflación, 2009-2023',
                  title_font = dict(color='#000000', size=30),
                  xaxis=dict(title='Año', titlefont_size=20, tickfont_size=20),
                  yaxis=dict(title='En porcentaje', titlefont_size=20, tickfont_size=18, linecolor="#000000", ticks='outside', nticks=10, zeroline=True, zerolinewidth=1.5, zerolinecolor='#d62f15'),
                  hoverlabel=dict(font_size=20,font_family="Arial"),
                  hovermode="x"
                  )

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df2.yearly, y=df2["Santa Cruz"], name="Depto_inflation_tarj",
                marker_color=df2.colors, marker_line=dict(width=0.5, color='#000000') ))
fig.update_layout(font_family = 'Arial', font_color="#000000", plot_bgcolor='white', separators=',',
                  title_text='Santa Cruz: Inflación, 2009-2023',
                  title_font = dict(color='#000000', size=30),
                  xaxis=dict(title='Año', titlefont_size=20, tickfont_size=20),
                  yaxis=dict(title='En porcentaje', titlefont_size=20, tickfont_size=18, linecolor="#000000", ticks='outside', nticks=10, zeroline=True, zerolinewidth=1.5, zerolinecolor='#d62f15'),
                  hoverlabel=dict(font_size=20,font_family="Arial"),
                  hovermode="x"
                  )

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df2.yearly, y=df2["Beni"], name="Depto_inflation_beni",
                marker_color=df2.colors, marker_line=dict(width=0.5, color='#000000') ))
fig.update_layout(font_family = 'Arial', font_color="#000000", plot_bgcolor='white', separators=',',
                  title_text='Beni: Inflación, 2009-2023',
                  title_font = dict(color='#000000', size=30),
                  xaxis=dict(title='Año', titlefont_size=20, tickfont_size=20),
                  yaxis=dict(title='En porcentaje', titlefont_size=20, tickfont_size=18, linecolor="#000000", ticks='outside', nticks=10, zeroline=True, zerolinewidth=1.5, zerolinecolor='#d62f15'),
                  hoverlabel=dict(font_size=20,font_family="Arial"),
                  hovermode="x"
                  )

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df2.yearly, y=df2["Pando"], name="Depto_inflation_pand",
                marker_color=df2.colors, marker_line=dict(width=0.5, color='#000000') ))
fig.update_layout(font_family = 'Arial', font_color="#000000", plot_bgcolor='white', separators=',',
                  title_text='Pando: Inflación, 2009-2023',
                  title_font = dict(color='#000000', size=30),
                  xaxis=dict(title='Año', titlefont_size=20, tickfont_size=20),
                  yaxis=dict(title='En porcentaje', titlefont_size=20, tickfont_size=18, linecolor="#000000", ticks='outside', nticks=10, zeroline=True, zerolinewidth=1.5, zerolinecolor='#d62f15'),
                  hoverlabel=dict(font_size=20,font_family="Arial"),
                  hovermode="x"
                  )

In [ ]:
import pandas as pd
import plotly.express as px

# Asumiendo que 'infla_depto' contiene los datos de inflación
#fig = px.area(infla_depto, x="mes", y=infla_depto.columns[1:], title="Inflación por Departamento")
fig = px.area(infla_depto, x="mes", y=infla_depto.columns[1:4], title="Inflación por Departamento") # Bolivia
# Agregar un slider para la fecha "mes"
fig.update_xaxes(rangeslider_visible=True)

# Personalizar etiquetas y leyenda, fondo blanco y tamaño de letra a 18
fig.update_layout(
    yaxis_title="En porcentaje",
    xaxis_title="Período",
    legend_title="Departamento",
    plot_bgcolor='white',  # Fondo blanco
    font=dict(family="LaTeX", size=18),  # Tipo de fuente en formato LaTeX y tamaño 18
    height=900,  # Ajustar la altura de la figura
    width=1200,   # Ajustar el ancho de la figura
    barmode='group'  # Configurar el modo de barras para que no sean acumuladas
)

# Mostrar el gráfico
fig.show()

In [ ]:
import pandas as pd
import plotly.express as px

# Asumiendo que 'infla_depto' contiene los datos de inflación
fig = px.area(depto_imes, x="mes", y=depto_imes.columns[1:], title="Inflación por Departamento")
# Agregar un slider para la fecha "mes"
fig.update_xaxes(rangeslider_visible=True)

# Personalizar etiquetas y leyenda, fondo blanco y tamaño de letra a 18
fig.update_layout(
    yaxis_title="En porcentaje",
    xaxis_title="Período",
    legend_title="Departamento",
    plot_bgcolor='white',  # Fondo blanco
    font=dict(family="LaTeX", size=18),  # Tipo de fuente en formato LaTeX y tamaño 18
    height=900,  # Ajustar la altura de la figura
    width=1200,   # Ajustar el ancho de la figura
    barmode='group'  # Configurar el modo de barras para que no sean acumuladas
)

# Mostrar el gráfico
fig.show()

In [ ]:
import pandas as pd
import plotly.express as px

# Asumiendo que 'depto_imes' contiene los datos de inflación
fig = px.area(depto_imes, x="mes", y=depto_imes.columns[1], title="Inflación por Departamento")

# Agregar un selector para elegir la variable
from ipywidgets import interact, widgets
from IPython.display import display

def actualizar_grafico(variable):
    fig.data[0].y = depto_imes[variable]
    fig.update_layout(title="Inflación por Departamento (" + variable + ")")
    display(fig)

opciones = depto_imes.columns[1:]
interact(actualizar_grafico, variable=widgets.Dropdown(options=opciones))


In [ ]:
import plotly.express as px

df = px.data.election()
geojson = px.data.election_geojson()

fig = px.choropleth_mapbox(df, geojson=geojson, color="Bergeron",
                           locations="district", featureidkey="properties.district",
                           center={"lat": 45.5517, "lon": -73.7073},
                           mapbox_style="carto-positron", zoom=9)
fig.show()

In [ ]:
!pip install -U plotly

# Gráfico Georreferenciado con Folium

In [ ]:
!pip install Folium

In [ ]:
import folium

#m = folium.Map(location=(-17.413977, -66.165322))
bolivia_coords = (-16.5000, -68.1193)
m = folium.Map(location=bolivia_coords, zoom_start=6.3)
m.save("index.html")
# Textura del
m = folium.Map((-16.5000, -68.1193), zoom_start=5, tiles="cartodb positron")

folium.Marker(
    location=[-15.568198, -68.274615],
    tooltip="La Paz",
    popup="2,1%",
    icon=folium.Icon(icon="cloud"),
).add_to(m)

folium.Marker(
    location=[-17.455822, -60.788391],
    tooltip="Santa Cruz",
    popup="3,4%",
    icon=folium.Icon(icon="home"),
).add_to(m)

folium.Marker(
    location=[-17.413977 , -66.165322],
    tooltip="Cochabamba",
    popup="2,6%",
    icon=folium.Icon(color="green"),
).add_to(m)


# Agrega marcadores para los departamentos
folium.Marker(
    location=(-13.696330, -65.480228),
    tooltip="Beni",
    popup="1,4%",
    icon=folium.Icon(color="star"),
).add_to(m)

folium.Marker(
    location=(-10.916330, -67.250471),
    tooltip="Pando",
    popup="2,2%",
    icon=folium.Icon(color="green"),
).add_to(m)

folium.Marker(
    location=(-18.510316, -67.697519),
    tooltip="Oruro",
    popup="0,9%",
    icon=folium.Icon(color="cyan"),
).add_to(m)

folium.Marker(
    location=(-20.487436, -66.930522),
    tooltip="Potosi",
    popup="2,9%",
    icon=folium.Icon(icon="plus"),
).add_to(m)

folium.Marker(
    location=(-21.536859, -64.729453),
    tooltip="Tarija",
    popup="1,2%",
    icon=folium.Icon(icon="heart"),
).add_to(m)

folium.Marker(
    location=(-19.033047, -65.262883),
    tooltip="Sucre",
    popup="0,8%",
    icon=folium.Icon(icon="cloud"),
).add_to(m)


# Añdir un línea entre punto de latitud y longitud
trail_coordinates = [
    (-16.647068, -68.092192),
    (-17.504905, -66.179033),
    (-17.891913, -63.176618),
]

folium.PolyLine(trail_coordinates, tooltip="Eje central").add_to(m)

m